In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as p
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import train_test_split
from folktables import *

In [3]:
import sys

## Folktables data

NOTE: please unzip file ./data/2018.zip

In [ ]:
ACSIncomeNew = folktables.BasicProblem(
    features=[
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'RELP',
        'WKHP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    group='SEX',
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

In [ ]:
def split_folk_data(state_one, state_two ):
    data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
    ca_data = data_source.get_data(states=[state_one], download=True)
    ca_features, ca_label, ca_group = ACSIncomeNew.df_to_numpy(ca_data)
    pa_data = data_source.get_data(states=[state_two], download=True)
    pa_features, pa_label, pa_group = ACSIncomeNew.df_to_numpy(pa_data)
    X_ls, X_calibrate, y_ls, y_calibrate = train_test_split(ca_features, ca_label, test_size=0.6, random_state=0)
    X_eval, X_eval2, y_eval, y_eval2 = train_test_split(pa_features, pa_label, test_size=0.6, random_state=0)

    # taking .2 of entire dataset for each fold for time
    X_ls, X_calibrate, y_ls, y_calibrate = train_test_split(X_ls, y_ls, test_size=0.5, random_state=0)
    X_eval, X_eval2, y_eval, y_eval2 = train_test_split(X_eval, y_eval, test_size=0.5, random_state=0)
    
    return X_ls, y_ls, X_calibrate, y_calibrate, X_eval, y_eval

In [ ]:
## folktable data params

dataset_name = 'Folktables'

# residual score normalization 
l = 0
u = 1217116

## Function definitions

In [ ]:
def train_qr(X_model, y_model, quant):
    qr = QuantileRegressor(quantile=quant, solver='highs')
    qr.fit(X_model, y_model)
    return qr

In [ ]:
def all_points(x):
    return True

def calibrate(x_input, y_input, bucket, r, delta, l, u, T, T_calibrate, myRLS):
    n = bucket
    r = r
        
    # no groups
    groups = [all_points]

    eta = 0.5
    delta = delta

    myUncertaintyQuantifier = MultiValidPrediction(delta, n, groups, eta, r)

    myResidualCalibrationScorer = residualCalibrationScorer.ResidualCalibrationScorer()

    myResidualCalibrationScorer.update(myRLS.predict)

    y_input = np.asarray(y_input)

    covered_arr = []
    width_arr = []

    for t in range(T):    

        x_t = np.matrix(x_input[t])
        y_t = y_input[t]

        # calculate the new threshold 
        norm_q_t = myUncertaintyQuantifier.predict(x_t)
        
        # rescale threshold
        q_t = norm_q_t * (u - l) + l

        # check if the prediction set covers the data
        curr_prediction_set = myResidualCalibrationScorer.get_prediction_set(x_t, q_t)
        covered_t = curr_prediction_set.cover(np.matrix(y_t))
        covered_arr.append(covered_t)

        # get prediction interval width
        width_arr.append(curr_prediction_set.interval_width)
        
        if (t > T_calibrate): # evaluate coverage / width after calibration data
            covered_arr.append(covered_t)
            width_arr.append(curr_prediction_set.interval_width)


        # update the calibrator mutlivalidator 
        s_t = myResidualCalibrationScorer.calc_score(x_t, np.matrix(y_t))

        # normalize score
        norm_s_t = (s_t - l ) / (u - l)

        myUncertaintyQuantifier.update(x_t, norm_q_t, norm_s_t)

        # update the calibration scorer
        myResidualCalibrationScorer.update(myRLS.predict)
    
    return covered_arr, width_arr

In [ ]:
def weighted_quantile(values, quantiles, sample_weight=None,   
    values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)


def do_weighted_conformal(x, y, weights, alpha, T, T_calibration, model):

    # arrays for conformal prediction
    width_conformal = []
    cover_conformal = []
    
    myResidualCalibrationScorer = residualCalibrationScorer.ResidualCalibrationScorer()

    y = np.asarray(y)


    for t in range(T):
        
        x_t = np.matrix(x[t])
        y_t = y[t]
        
        y_pred_conformal_t = model.predict(x_t)
        myResidualCalibrationScorer.update(model.predict)
        new_y = np.reshape(y, -1)
        residuals = myResidualCalibrationScorer.calc_score(x, new_y)
        calibration_size = len(x)

        desired_quantile = np.ceil((1-alpha) * (calibration_size + 1)) / calibration_size
        
        w_t_conformal = weighted_quantile(residuals, (1-alpha), weights)

        conformal_prediction_set = myResidualCalibrationScorer.get_prediction_set(x_t, w_t_conformal)

        covered_conformal_t = conformal_prediction_set.cover(y_t)

        if (t > T_calibration): # evaluate coverage / width after calibration data
            width_conformal.append(w_t_conformal)
            cover_conformal.append(covered_conformal_t)

    return width_conformal, cover_conformal

## Experimental parameters

In [ ]:
num_trials = 1
buckets = [40]
d = 9

# 1 - coverage
alpha = .1

model_name = 'Quantile Regression'

## Experiments

In [ ]:
sys.path.append('../src')
from MultiValidPrediction import MultiValidPrediction
from calibrationScorers import residualCalibrationScorer
import recursiveLeastSquares

In [ ]:
# for each setting of number of buckets, we have num_trials coverage/width values
folk_coverage = []
folk_width = []

folk_coverage_conf = []
folk_width_conf = []

In [ ]:
X_ls, y_ls, X_calibrate, y_calibrate, X_eval, y_eval = split_folk_data('CA', 'PA')

In [ ]:
conformal = True
for num_bucket in buckets:
    
    trial_coverage = []
    trial_width = []
    trial_cov_conf = []
    trial_width_conf = []
    
    for i in range(num_trials):
        # split data
        X_ls, y_ls, X_calibrate, y_calibrate, X_eval, y_eval = split_folk_data('CA', 'PA')

        # train model
        reg_model = train_qr(X_ls, y_ls, .5)

        X = np.concatenate([X_calibrate, X_eval])
        y = np.concatenate([y_calibrate, y_eval])
        
        # calibrate
        coverage_res, width_res = calibrate(X, y, num_bucket, 80000, alpha, l, u, len(X), len(X_calibrate), reg_model)

        # store average coverage and width for this trial
        trial_coverage.append(np.mean(coverage_res))
        trial_width.append(np.mean(width_res))
        
        # conformal
        if conformal: # only do conformal once, not for each different setting of number of buckets
            
            # split conformal on shifted data
            conf_width_res, conf_cov_res = do_weighted_conformal(X, y, (([1] * len(X))), alpha, len(X), len(X_calibrate), reg_model)

            trial_cov_conf.append(np.mean(conf_cov_res))
            trial_width_conf.append(np.mean(conf_width_res))


    if conformal:
        folk_coverage_conf.append(trial_cov_conf)
        folk_width_conf.append(trial_width_conf)
        conformal = False
        
    folk_coverage.append(trial_coverage)
    folk_width.append(trial_width)

In [ ]:
plt.hist([folk_coverage[0], folk_coverage_conf[0]], label=['MVP', 'split conformal'])
plt.legend()
plt.title('Mean Coverage ({0} trials, target coverage .9) \n {1} \n {2}'.format(len(folk_coverage[0]), model_name, dataset_name))
plt.ylabel('Frequency')
plt.xlabel('Coverage')
plt.show()

plt.hist([folk_width[0], folk_width_conf[0]], label=['MVP', 'split conformal'])
plt.legend()
plt.title('Median Width ({0} trials, target coverage .9) \n {1} \n {2}'.format(len(folk_coverage[0]), model_name, dataset_name))
plt.ylabel('Frequency')
plt.xlabel('Interval Width (in $)')
plt.show()

## Plotting

In [ ]:
fd = open('folk-results/folk-coverage-conformal.pickle', 'rb')
folk_coverage_conformal = p.load(fd)
fd.close()

fd = open('folk-results/folk-coverage.pickle', 'rb')
folk_coverage = p.load(fd)
fd.close()

fd = open('folk-results/folk-width-conformal.pickle', 'rb')
folk_width_conformal = p.load(fd)
fd.close()

fd = open('folk-results/folk-width.pickle', 'rb')
folk_width = p.load(fd)
fd.close()

In [ ]:
model_name = 'Quantile Regression'
dataset_name = 'Folktables Data'

In [ ]:
bins = np.linspace(0.87, 0.95, 60)
plt.hist([folk_coverage[0]], label=['MVP'], color= 'blue', alpha = 0.4, linewidth=0.5, edgecolor = 'blue', bins=bins)
plt.hist([folk_coverage_conformal[0]], label=['split conformal'], color= 'orange', alpha = 0.6, linewidth=0.5, edgecolor = 'orange', bins=bins)
plt.legend()
plt.axvline(x = .9, color = 'red', linestyle = '--', linewidth = 0.9)
plt.title('Mean Coverage ({0} trials, target coverage .9) \n {1} \n {2}'.format(len(folk_coverage[0]), model_name, dataset_name))
plt.ylabel('No. of Rounds')
plt.xlabel('Mean Coverage')
plt.xlim([.899, .96])
plt.show()

bins = np.linspace(72700, 73500, 60)
plt.hist([folk_width[0]], label=['MVP'], color= 'blue', alpha = 0.4, linewidth=0.5, edgecolor = 'blue', bins=bins)
plt.hist([folk_width_conformal[0]], label=['split conformal'], color= 'orange', alpha = 0.6, linewidth=0.5, edgecolor = 'orange', bins=bins)
plt.legend()
plt.legend()
plt.title('Median Width ({0} trials, target coverage .9) \n {1} \n {2}'.format(len(folk_coverage[0]), model_name, dataset_name))
plt.ylabel('No. Of Rounds')
plt.xlabel('Median Interval Width (in $)')
plt.show()